In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch
from torch import optim

In [14]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

data = load_dataset("csv", data_files = "../data/cleaned/nlp_clean.csv", sep = ",", encoding='ISO-8859-1')
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


Found cached dataset csv (C:/Users/ANSD/.cache/huggingface/datasets/csv/default-fd58bde105b4d41e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = data.map(tokenize_function, batched=True)


Loading cached processed dataset at C:\Users\ANSD\.cache\huggingface\datasets\csv\default-fd58bde105b4d41e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-aa5ce311a92211b8.arrow


In [16]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.remove_columns(["Unnamed: 0.1"])
tokenized_datasets = tokenized_datasets.remove_columns(["Unnamed: 0"])
#tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

small_train_dataset = tokenized_datasets["train"].shuffle().select(range(5000))
small_eval_dataset = tokenized_datasets["train"].shuffle().select(range(5000))


In [17]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)


In [18]:
small_train_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5000
})

In [19]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [20]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)


In [21]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [22]:
"""

for batch in train_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

"""

'\n\nfor batch in train_dataloader:\n    batch = {k: v.to(device) for k, v in batch.items()}\n    outputs = model(**batch)\n    loss = outputs.loss\n    loss.backward()\n    optimizer.step()\n    optimizer.zero_grad()\n\n'

In [23]:
import torch
# Import other necessary libraries

# Define other training parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 5
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  # Define your optimizer
model.train()

for epoch in range(epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        optimizer.zero_grad()
        #outputs = model(inputs, attention_mask=attention_mask)  # Pass attention_mask if applicable
        loss = outputs.loss  # Make sure your model's output dictionary contains 'loss'
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

    # Add validation/test phase here

KeyboardInterrupt: 

In [ ]:
# Load and preprocess the CSV data
data = pd.read_csv("../data/cleaned/nlp_clean.csv")  # Replace with your file path


In [ ]:
data = data.dropna()

In [ ]:
data.to_csv("../data/cleaned/nlp_clean.csv")

In [ ]:
data.isna().sum()

Unnamed: 0    0
text          0
labels        0
dtype: int64

In [ ]:
texts = data["text"].tolist()
labels = data["labels"].tolist()

In [ ]:
len(texts) , len(labels)

(41157, 41157)

In [ ]:
# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)


In [ ]:
# Load pre-trained transformer model and tokenizer
model_name = "distilbert-base-uncased"  # Replace with your desired pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)  # Assuming 5 sentiment classes


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [ ]:
# Tokenize the input data
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./models",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./docs",
    evaluation_strategy="epoch",
    logging_steps=500,
    save_steps=2000,
    learning_rate=2e-5,
    save_total_limit=1,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# Define the Trainer for fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encodings,
    eval_dataset=val_encodings,
    compute_metrics=lambda p: {"accuracy": accuracy_score(p.predictions.argmax(axis=1), p.label_ids)},
)

In [ ]:
# Fine-tune the model
trainer.train()

# Evaluate the model on the validation set
results = trainer.evaluate()

print("Results:", results)

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

c:\Users\ANSD\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3
  Number of trainable parameters = 66957317


  0%|          | 0/3 [00:00<?, ?it/s]

ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,attention_mask,head_mask,inputs_embeds,labels,output_attentions,output_hidden_states,return_dict,label,label_ids,labels.

In [ ]:
class Mydataset(Dataset):
  def __init__(self, df, tokenizer_name, max_length):
    self.max_length = max_length
    self.df = df
    self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
  def __len__(self):
    return len(self.df)
  def __getitem__(self, index):
    text = self.df["text"][index]
    label = self.df["labels"][index]
    #inputs = self.tokenizer(text = text, padding = True, max_length = self.max_length, truncation = True, return_tensors = 'pt')
    inputs = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')

    return {
        "input_ids" : inputs["input_ids"],
        "attention_mask" : inputs["attention_mask"],
        "labels" : torch.tensor(label)
    }

In [ ]:
myData = Mydataset(data, tokenizer_name="distilbert-base-uncased", max_length=128)

In [ ]:
def dataLoader(dataset, batch_size, shuffle):
  return DataLoader(dataset = dataset, batch_size = batch_size, shuffle = shuffle)

In [ ]:
data_ready = dataLoader(dataset=myData, batch_size=5, shuffle=True)

In [ ]:
next(iter(data_ready))

{'input_ids': tensor([[[  101,  5472,  4381,  3215,  2017,  2342,  2000,  2644,  1996,  2235,
            6240,  7340,  1998,  2789,  7340,  1998,  2015,  1998, 11085,  2389,
            7340,  1999,  2225, 27888, 13039,  2045,  7597,  2039,  2335,  2062,
            1996,  2434,  3976,  2138,  1997,  2522, 17258,  2742,  1999,  4497,
            2379,  5472,  4381,  2473,  2457,  2160,  5785, 25269,  2361, 16215,
             102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=1e-5)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {key: val.to(device) for key, val in inputs.items()}
labels = labels.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5)

model.train()
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(**inputs, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# Save the fine-tuned model
model.save_pretrained("fine_tuned_model")

AttributeError: 'list' object has no attribute 'to'

In [ ]:
help(model)

Help on DistilBertForSequenceClassification in module transformers.models.distilbert.modeling_distilbert object:

class DistilBertForSequenceClassification(DistilBertPreTrainedModel)
 |  DistilBertForSequenceClassification(config: transformers.configuration_utils.PretrainedConfig)
 |  
 |  DistilBert Model transformer with a sequence classification/regression head on top (a linear layer on top of the
 |  pooled output) e.g. for GLUE tasks.
 |  
 |  
 |  This model inherits from [`PreTrainedModel`]. Check the superclass documentation for the generic methods the
 |  library implements for all its model (such as downloading or saving, resizing the input embeddings, pruning heads
 |  etc.)
 |  
 |  This model is also a PyTorch [torch.nn.Module](https://pytorch.org/docs/stable/nn.html#torch.nn.Module) subclass.
 |  Use it as a regular PyTorch Module and refer to the PyTorch documentation for all matter related to general usage
 |  and behavior.
 |  
 |  Parameters:
 |      config ([`DistilB

In [ ]:
for batch in data_ready:
    inputs = {key: val.to(device) for key, val in batch.items() if key not in ["labels"]}
    inputs

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
model()

In [ ]:
# training loop
# Define other training parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 5
model.train()
for batch in data_ready:
  inputs = {key: val.to(device) for key, val in batch.items() if key not in ["labels"]}
  labels = batch["labels"].to(device)
  optimizer.zero_grad()
  outputs = model(**inputs)
  loss = outputs.loss
  loss.backward()
  optimizer.step()

print(f"Epoch {epoch+1}, Loss: {loss.item()}")

ValueError: too many values to unpack (expected 3)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./sentiment_model",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    evaluation_strategy="epoch",
    logging_steps=500,
    save_steps=2000,
    learning_rate=2e-5,
    save_total_limit=1,
)

# Define the Trainer for fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encodings,
    eval_dataset=val_encodings,
    compute_metrics=lambda p: {"accuracy": accuracy_score(p.predictions.argmax(axis=1), p.label_ids)},
)

# Fine-tune the model
trainer.train()

# Evaluate the model on the validation set
results = trainer.evaluate()

print("Results:", results)

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")


NameError: name 'train_encodings' is not defined